<a target="_blank" href="https://colab.research.google.com/github/Datacompintensive/LLFT/blob/master/Notebooks/Coffee2/Coffee.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
from datetime import datetime

In [2]:
try:
    from google.colab import drive
    IN_COLAB = True
except:
    IN_COLAB = False
print(f'Running on {"Google colab" if IN_COLAB else "Local computer"}')

Running on Google colab


In [3]:
if IN_COLAB:
    !pip install scikit-optimize
    !pip install aeon
    try:
        !rm -rf LLFT.py
        !rm -rf ClassificationLearner.py
    except Exception as e:
        print(e)
    !wget https://raw.githubusercontent.com/Datacompintensive/LLFT/master/src/LLFT.py
    !wget https://raw.githubusercontent.com/Datacompintensive/LLFT/master/src/ClassificationLearner.py
    from LLFT import *
    from ClassificationLearner import *
    drive.mount('/content/drive/')
    #Overwrite path
    %cd /content/drive/My Drive/Research/Wigner/TimeSeries
else:
    %load_ext autoreload
    %autoreload 2
    # Driver magic
    import sys
    sys.path.insert(0, "../../src")
    from LLFT import *
    from ClassificationLearner import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 862.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 MB 10.3 MB/s eta 0:00:00
--2024-06-12 20:18:08--  https://raw.githubusercontent.com/Datacompintensive/LLFT/master/src/LLFT.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14093 (14K) [text/plain]
Saving to: ‘LLFT.py’

LLFT.py             100%[===================>]  13.76K  --.-KB/s    in 0s      

2024-06-12 20:18:08 (71.0 MB/s) - ‘LLFT.py’ saved [14093/14093]

--2024-06-12 20:18:08--  https://raw.githubusercontent.com/Datacompintensive/LLFT/master/src/ClassificationLearner.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.git

In [4]:
database = "ElectricDevices"

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [6]:
seed = 0

In [ ]:
if IN_COLAB:
    from aeon.datasets import load_classification
    X, y = load_classification(database)
    y = y.astype(np.int8)
    #CHANGE HERE!
    train_set, test_set = X[:28], X[28:]
    train_classes, test_classes = y[:28], y[28:]
else:
    dbpath = f"../../databases/{database}/"

    X = pd.read_csv(f"{dbpath}{database}_TEST", header=None).values
    test_classes = X[:,0]
    test_set = X[:,1:]

    X = pd.read_csv(f"{dbpath}{database}_TRAIN", header=None).values
    train_classes = X[:,0]
    train_set = X[:,1:]

test_classes = torch.tensor(test_classes, dtype=torch.int8)
test_set = torch.tensor(test_set, dtype=torch.float32)
train_classes = torch.tensor(train_classes, dtype=torch.int8)
train_set = torch.tensor(train_set, dtype=torch.float32)

train_set, c_train_set, train_classes, c_train_classes = train_test_split(train_set, train_classes, train_size=0.5, random_state=seed)



print(f"train_set shape: {train_set.shape}")
print("train classes:", train_classes.unique(return_counts=True))
print()
print(f"c_train_set shape: {c_train_set.shape}")
print("c train classes:", c_train_classes.unique(return_counts=True))
print()
print(f"test_set shape: {test_set.shape}")
print("test classes:", test_classes.unique(return_counts=True))

train_set shape: torch.Size([4463, 96])
train classes: (tensor([1, 2, 3, 4, 5, 6, 7], dtype=torch.int8), tensor([ 361, 1101,  447,  713, 1193,  269,  379]))

c_train_set shape: torch.Size([4463, 96])
c train classes: (tensor([1, 2, 3, 4, 5, 6, 7], dtype=torch.int8), tensor([ 366, 1130,  404,  761, 1213,  240,  349]))

test_set shape: torch.Size([7711, 96])
test classes: (tensor([1, 2, 3, 4, 5, 6, 7], dtype=torch.int8), tensor([ 667, 1956,  755, 1165, 1869,  743,  556]))


In [ ]:
train_set.shape[-1] / 2

48.0

In [ ]:
L = [20]
k = 1

llft = LLFT(train_set, train_classes, L=L, k=k, device=device)
llft.train()

In [7]:
extr_methods = {ExtractMethods.mean : {}, ExtractMethods.var : {}}
c_train_filename = f"{database}_c_features.csv"
test_filename = f"{database}_test_features.csv"

In [ ]:
transformed_test_set = llft.collect_features(test_set, extr_methods=extr_methods, test_classes=test_classes,\
                                             save_file_name=test_filename, save_file_mode="Append feature")
display(transformed_test_set.shape)

torch.Size([7711, 14])

In [ ]:
transformed_c_train_set = llft.collect_features(c_train_set, extr_methods=extr_methods, test_classes=c_train_classes,\
                                                save_file_name=c_train_filename, save_file_mode="Append feature")
display(transformed_c_train_set.shape)

torch.Size([4463, 14])

In [ ]:
L_vals = [8, 12, 25, 30, 35, 40, 45, 48]
for L in L_vals:
    try:
        print(f"L: {L}")
        k = 1

        llft = LLFT(train_set, train_classes, L=L, k=k, device=device)
        llft.train()
        transformed_test_set = llft.collect_features(test_set, extr_methods=extr_methods, test_classes=test_classes,\
                                                save_file_name=test_filename, save_file_mode="Append feature")
        display(transformed_test_set.shape)
        transformed_c_train_set = llft.collect_features(c_train_set, extr_methods=extr_methods, test_classes=c_train_classes,\
                                                    save_file_name=c_train_filename, save_file_mode="Append feature")
        display(transformed_c_train_set.shape)
    except Exception as e:
        print(f"Error: {e}")

L: 8
Error: linalg.eigh: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated eigenvalues (error code: 1).
L: 12
Error: linalg.eigh: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated eigenvalues (error code: 1).
L: 25


torch.Size([7711, 14])

torch.Size([4463, 14])

L: 30


torch.Size([7711, 14])

...

torch.Size([7711, 14])

torch.Size([4463, 14])

L: 48


torch.Size([7711, 14])

torch.Size([4463, 14])

In [ ]:
display(pd.read_csv(c_train_filename).values.shape)
display(pd.read_csv(test_filename).values.shape)

(4463, 127)

(7711, 127)

In [ ]:
used_classes = None
verbose=True
cl = ClassificationLearner(transformed_c_train_set, transformed_c_train_classes, used_classes=used_classes, validation_size=0.1, verbose=verbose, random_state=0)
cl.train(["opt-knn"], bayes_settings={"n_iter": 10}, verbose=verbose)
cl.test(["all"], transformed_test_set, transformed_test_classes, verbose=verbose)

X_train shape: (4016, 126)
Train classes: (array([1., 2., 3., 4., 5., 6., 7.]), array([ 337, 1004,  365,  703, 1090,  215,  302]))

X_validation shape: (447, 126)
Validation classes: (array([1., 2., 3., 4., 5., 6., 7.]), array([ 29, 126,  39,  58, 123,  25,  47]))

opt-knn optimization; validation:
Method: opt-knn
Accuracy: 0.69575
Confusion Matrix:
[[ 23   0   1   4   0   1   0]
 [  0 120   0   1   5   0   0]
 [  0   0  28   3   2   0   6]
 [  7   0   6  29   9   6   1]
 [  3   7   3  10  94   2   4]
 [  7   0   1   8   0   9   0]
 [  4   1  18   3  12   1   8]]


Method: def-knn
Accuracy: 0.56932
Confusion Matrix:
[[ 249    0   13  179  160   64    2]
 [   6 1448   38  205  255    0    4]
 [  27    0  556   41   14    9  108]
 [ 159    7  129  616  143   74   37]
 [  58  218   47  150 1356    6   34]
 [ 219    0   23  323   25  149    4]
 [   8  109    8   70  345    0   16]]

Method: def-svm
Accuracy: 0.55518
Confusion Matrix:
[[ 205  145   36  266   15    0    0]
 [   3 1753   32  

[[0.6822720788483984,
  'def-rf',
  {'bootstrap': True,
   'ccp_alpha': 0.0,
   'class_weight': None,
   'criterion': 'gini',
   'max_depth': None,
   'max_features': 'sqrt',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'n_estimators': 100,
   'n_jobs': None,
   'oob_score': False,
   'random_state': None,
   'verbose': 0,
   'warm_start': False}],
 [0.5995331344832058,
  'def-dt',
  {'ccp_alpha': 0.0,
   'class_weight': None,
   'criterion': 'gini',
   'max_depth': None,
   'max_features': None,
   'max_leaf_nodes': None,
   'min_impurity_decrease': 0.0,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'random_state': None,
   'splitter': 'best'}],
 [0.5905848787446505,
  'opt-knn',
  {'algorithm': 'ball_tree',
   'leaf_size': 30,
   'metric': 'manhattan',
   'metric_params': None,
   'n_jobs': None,
   'n_neighbo

In [15]:
from skopt import gp_minimize

class HyperparameterOptimizer:
    def __init__(self, c_train_filename, test_filename, verbose=True, random_state=0):
        self.verbose = verbose
        self.random_state = random_state
        np.random.seed(random_state)
        transformed_c_train_data = pd.read_csv(c_train_filename).values
        self.transformed_c_train_set, self.transformed_c_train_classes = transformed_c_train_data[:, :-1], transformed_c_train_data[:, -1]

        transformed_test_data = pd.read_csv(test_filename).values
        self.transformed_test_set, self.transformed_test_classes = transformed_test_data[:, :-1], transformed_test_data[:, -1]

        self.headers = list(pd.read_csv(c_train_filename))
        self.L_array, self.C_array, self.F_array, self.m_array = self.extract_and_process_parts(self.headers[:-1])

        if self.verbose:
            print(f"Headers: {self.headers[:5]}..., length: {len(self.headers)}")
            print(f"L: {self.L_array}, C: {self.C_array}, F: {self.F_array}, m: {self.m_array}")
            print()
            print(f"transformed_c_train_set.shape: {self.transformed_c_train_set.shape}")
            print(f"transformed_c_train_classes: {np.unique(self.transformed_c_train_classes, return_counts=True)}")
            print()
            print(f"transformed_test_set.shape: {self.transformed_test_set.shape}")
            print(f"transformed_test_classes: {np.unique(self.transformed_test_classes, return_counts=True)}")


    def extract_and_process_parts(self, items):
        # Extract parts and convert them to integers where applicable
        L_values, C_values, F_values, m_values = zip(*[(int(L[1:]), int(C[1:]), F[1:], int(m[1:])) for L, C, F, m in (item.split('-') for item in items)])

        # Convert lists to numpy arrays
        L_array = np.unique(np.array(L_values))
        C_array = np.unique(np.array(C_values))
        F_array = list(np.unique(np.array(F_values)))
        m_array = np.unique(np.array(m_values))

        return L_array, C_array, F_array, m_array


    def find_indices(self, main_list, subset_list):
        """
        Finds the indices of the elements of subset_list within main_list.

        Parameters:
        main_list (list): The main list containing all elements.
        subset_list (list): The subset list whose indices in the main list are to be found.

        Returns:
        list: A list of indices corresponding to the positions of subset_list elements in main_list.
        """

        # Check for non-unique values in the main_list
        seen = set()
        duplicates = set()
        for element in main_list:
            if element in seen:
                duplicates.add(element)
            else:
                seen.add(element)
        if duplicates:
            print(f"Error: Non-unique values found in main_list: {duplicates}")

        indices = []
        for element in subset_list:
            try:
                index = main_list.index(element)
                indices.append(index)
            except ValueError:
                print(f"Element {element} not found in main_list.")

        return indices

    def get_headers(self, L=None, C=None, F=None, m=None, return_indices=True):

        if L and type(L) is not list:
            L = [L]
        if C and type(C) is not list:
            C = [C]
        if F and type(F) is not list:
            F = [F]
        if m and type(m) is not list:
            m = [m]

        ret = self.headers[:]
        if L:
            tmp = []
            for l in L:
                tmp += [x for x in ret if f"L{l}-" in x]
            ret = tmp[:]
        if C:
            tmp = []
            for c in C:
                tmp += [x for x in ret if f"C{c}-" in x]
            ret = tmp[:]
        if F:
            tmp = []
            for f in F:
                tmp += [x for x in ret if f"F{f}-" in x]
            ret = tmp[:]
        if m:
            tmp = []
            for M in m:
                tmp += [x for x in ret if f"m{M}" in x]
            ret = tmp[:]

        if return_indices:
            ret = self.find_indices(self.headers, ret)
        return ret

    def objective_function(self, subset_L, subset_F):
        cl = self.create_cl_object(subset_L, subset_F)
        ret = cl.test(["all"], self.transformed_test_set, self.transformed_test_classes, verbose=False)
        #ret = cl.train(["all-def"], verbose=False)
        if self.verbose:
            print(f"L: {subset_L}, F: {subset_F}, accuracy: {ret[0][0:2]}")
        return ret[0][0]

    def bayesian_optimization(self, n_iterations=20):
        list1, list2 = self.L_array, self.F_array
        def wrapped_objective(params):
            subset_indices1 = params[:len(list1)]
            subset_indices2 = params[len(list1):]

            subset1 = [list1[i] for i in range(len(list1)) if subset_indices1[i]]
            subset2 = [list2[i] for i in range(len(list2)) if subset_indices2[i]]

            return -self.objective_function(subset1, subset2)  # We negate because we want to maximize

        dimensions = [Categorical([0, 1]) for _ in range(len(list1) + len(list2))]

        result = gp_minimize(wrapped_objective, dimensions, n_calls=n_iterations, random_state=self.random_state)

        best_indices = result.x
        best_subset1 = [list1[i] for i in range(len(list1)) if best_indices[i]]
        best_subset2 = [list2[i] for i in range(len(list2)) if best_indices[len(list1) + i]]

        cl = self.create_cl_object(best_subset1, best_subset2, False)
        cl_test = cl.test(["all"], self.transformed_test_set, self.transformed_test_classes, verbose=True)
        return best_subset1, best_subset2, -result.fun, cl_test

    def random_search(self, n_iterations=20):
        list1, list2 = self.L_array, self.F_array
        best_value = -float('inf')
        best_subset1 = []
        best_subset2 = []

        for _ in range(n_iterations):
            subset1 = [list1[i] for i in range(len(list1)) if np.random.choice([True, False])]
            subset2 = [list2[i] for i in range(len(list2)) if np.random.choice([True, False])]

            if subset1 and subset2:
                value = self.objective_function(subset1, subset2)
                if value > best_value:
                    best_value = value
                    best_subset1 = subset1
                    best_subset2 = subset2

        cl = self.create_cl_object(best_subset1, best_subset2, False)
        cl_test = cl.test(["all"], self.transformed_test_set, self.transformed_test_classes, verbose=True)
        return best_subset1, best_subset2, best_value, cl_test

    def create_cl_object(self, subset_L, subset_F, verbose=False):
        used_classes = self.get_headers(L=subset_L, F=subset_F, return_indices=True)
        cl = ClassificationLearner(self.transformed_c_train_set, self.transformed_c_train_classes, used_classes=used_classes, validation_size=0.1, verbose=verbose, random_state=self.random_state)
        #cl.train(["opt-knn"], bayes_settings={"n_iter": 10}, verbose=False)
        return cl

c_train_filename = f"Data/{database}_c_features.csv"
test_filename = f"Data/{database}_test_features.csv"

ho = HyperparameterOptimizer(c_train_filename, test_filename, True)

print("\nRandom Search:")
# Random Search
best_subset1_rs, best_subset2_rs, best_value_rs, cl_test = ho.random_search(n_iterations=3)
print("Best Subset 1:", best_subset1_rs)
print("Best Subset 2:", best_subset2_rs)
print("Best Objective Value:", best_value_rs)
print("Cl test", cl_test[0][0:2])

print()
print("Bayesian Optimization:")
# Bayesian Optimization
best_subset1_bo, best_subset2_bo, best_value_bo, cl_test = ho.bayesian_optimization(n_iterations=10)
print("Best Subset 1:", best_subset1_bo)
print("Best Subset 2:", best_subset2_bo)
print("Best Objective Value:", best_value_bo)
print("Cl test", cl_test)

Headers: ['L2-C0-Fmean-m0', 'L2-C0-Fvar-m0', 'L2-C1-Fmean-m0', 'L2-C1-Fvar-m0', 'L2-C2-Fmean-m0']..., length: 127
L: [ 2  5 20 25 30 35 40 45 48], C: [0 1 2 3 4 5 6], F: ['mean', 'var'], m: [0]

transformed_c_train_set.shape: (4463, 126)
transformed_c_train_classes: (array([1., 2., 3., 4., 5., 6., 7.]), array([ 366, 1130,  404,  761, 1213,  240,  349]))

transformed_test_set.shape: (7711, 126)
transformed_test_classes: (array([1., 2., 3., 4., 5., 6., 7.]), array([ 667, 1956,  755, 1165, 1869,  743,  556]))

Random Search:
L: [2, 5, 25, 30, 35, 40, 45], F: ['mean'], accuracy: [0.6744909869018285, 'def-rf']
L: [5, 40], F: ['mean', 'var'], accuracy: [0.6372714304240695, 'def-rf']
Method: def-knn
Accuracy: 0.56011
Confusion Matrix:
[[ 235    3    6  159  163  101    0]
 [   1 1360   20  153  421    1    0]
 [  10    4  552   46   15   12  116]
 [ 143    3  140  600  155  101   23]
 [  26  274   32  180 1334    4   19]
 [ 154    1   12  321   37  217    1]
 [   0  199    6   40  289    1   